In [ ]:
import pandas as pd
import json
import seaborn as sns
import operator

### Подготовка списка фильмов с ключевыми словами

##### Подготовка датафрейма с метаданными фильмов (без ключевых слов)

In [ ]:
# загрузка исходного датафрейма с метаданными фильмов, удаление пустых и ложных записей, удаление дупликатов, приведение к типам

df_meta = pd.read_csv('./moviesdataset/movies_metadata.csv',low_memory=False)
df_meta = df_meta.loc[:,['title','id','imdb_id']]


df_meta = df_meta.dropna(subset='imdb_id')
df_meta = df_meta.drop_duplicates(subset='imdb_id')
df_meta = df_meta.drop_duplicates(subset='id')
df_meta = df_meta.drop(df_meta[df_meta['id'].str.contains('-')].index)
df_meta['imdb_id'] = df_meta['imdb_id'].apply(lambda x: x.replace('tt',''))


df_meta = df_meta.astype({'title': 'string', 'id': 'int32','imdb_id': 'int32'})
df_meta = df_meta.set_index('id')

df_meta.dtypes
df_meta.shape

##### Подготовка датафрейма с ключевыми словами

In [ ]:
# загрузка исходного датафрейма с ключевыми словами, приведение к типам

df_kw = pd.read_csv('./moviesdataset/keywords.csv')
df_kw = df_kw.astype({'id': 'int32','keywords': 'string'})
df_kw = df_kw.drop_duplicates(subset='id')

df_kw = df_kw.set_index('id')
df_kw

In [ ]:
# обращение ключевых слов dict -> list для удобства работы, подчёт кол-ва ключевых слов в каждом фильме

def get_kw_list(data):
    
    kw_list = [kw['name'].replace(" ","").replace("\xa0","").replace("'","") for kw in eval(data)]
    return kw_list

df_kw.loc[:,'keylist'] = df_kw['keywords'].apply(get_kw_list)
df_kw['keywords_len'] = df_kw['keylist'].apply(len)
# df_kw = df_kw.drop(['keywords'],axis=1)
# df_kw.iloc[0,:]['keylist']
# df_kw = df_kw.iloc[:,:][df_kw['keywords_len'] >= 15]
df_kw

In [ ]:
# удаление строк без ключевых слов

df_kw = df_kw[df_kw['keylist'].apply(len) != 0]

df_kw

In [ ]:
# подготовка частотного словаря ключевых слов
def get_kw_counts(column):

    kw_series = pd.Series([i for j in column.tolist() for i in j])

    kw_count = kw_series.value_counts()

    counts = pd.DataFrame({'kw':kw_count.index,'count': kw_count})

    counts = counts.drop_duplicates(subset='kw')
    counts = counts.sort_values(by=['count','kw'], ascending=[False, True])

    kw_quantity = len(counts)
    will_deleted = len(counts[counts['count']==1])

    print(f'Ключевых слов, встречающихся однажды: {will_deleted} ({round(will_deleted/kw_quantity*100,3)}%)')

    return counts

counts = get_kw_counts(df_kw['keylist'])
counts

In [ ]:
def print_kw_stat(column):
    counts = get_kw_counts(column)
    most_popular_count = counts.iloc[0,:]['count']
    most_popular_kw = counts.iloc[0,:]['kw']
    most_rare_count = counts.iloc[-1,:]['count']
    most_rare_kw = counts.iloc[-1,:]['kw']
    print(f'Самое популярное слово: {most_popular_kw}, {most_popular_count} раз')
    print(f'Самое редкое слово: {most_rare_kw}, {most_rare_count} раз')
    print(f'Всего слов: {len(counts)}')

print_kw_stat(df_kw['keylist'])

In [ ]:
# удаление тех ключевых слов, которые встречаются только однажды

counts_first = get_kw_counts(df_kw['keylist'])
onlyonemeet_first = counts_first[counts_first['count'] == 1]['kw']

def kw_del_onlyonemeet(keylist):

    kw_list = []

    for kw in keylist:
        if kw in onlyonemeet_first:
            pass
        else:
            kw_list.append(kw)
    
    return kw_list

df = df_kw.copy(deep=True)
df['keylist_nolonely'] = df.loc[:,'keylist'].apply(kw_del_onlyonemeet)

counts_second = get_kw_counts(df['keylist_nolonely'])

In [ ]:
# отбор не более 15 ключевых слов (удалением наиболее редких)

counts = get_kw_counts(df['keylist_nolonely'])
quantity = 15

def get_most_common(keylist):

    kw_list = keylist

    if len(kw_list) > quantity:

        kw_cleaned = {}

        for kw in kw_list:
            kw_cleaned[kw] = counts[counts['kw'] == kw]['count'].values[0]

        most_common = [kw[0] for kw in sorted(kw_cleaned.items(), key=operator.itemgetter(1), reverse=True)[:quantity]]
        
        return most_common

    else:
        return keylist   

df['keylist_fifteen'] = df['keylist_nolonely'].apply(get_most_common)
df['keywords_len_commons'] = df['keylist_fifteen'].apply(len)
df

In [ ]:
# отбор не менее 15 слов 

df = df[df['keywords_len_commons'] == 15]

In [ ]:
# объединение с датасетом с метаданными (для получения названий)
df_joined = df.join(df_meta, on='id')
df_joined.loc[:,['keylist_fifteen','title','imdb_id']]

In [ ]:
df_joined = df_joined.set_index('imdb_id')

In [ ]:
df_subloaded = pd.read_csv('fil_sub_download_temp_file.csv', sep=';', names=['imdb_id','moviename', 'data','file'])
df_subloaded = df_subloaded.astype({'imdb_id': 'int32','moviename': 'string','data': 'object','file': 'object'})
df_subloaded = df_subloaded.drop_duplicates(subset='imdb_id')
df_subloaded = df_subloaded.drop(df_subloaded[df_subloaded['moviename'] == 'ERROR'].index)
df_subloaded = df_subloaded.set_index('imdb_id')
df_subloaded

In [ ]:
df_doned = df_subloaded.join(df_joined, on='imdb_id', how='left')


In [ ]:
df_doned = df_doned.loc[:,['keylist_fifteen']]
df_doned.dropna(inplace=True)

In [ ]:
df_doned

In [ ]:
df_doned.to_csv('df_keylist_fifteen.csv', sep=';')